## Objective

To analyze the second-level effect, on how the location of customer city affects the delivery time

Modeling: multi-level GLMM (with the use of some location features of customer cities as the second-level features).
This model is then compared with the base model (GLMM as proposed in analysis1).

In [7]:
library(R2WinBUGS)
library(dplyr)
library(readr)

#The directory of the WinBUGS
WinBUGS_path = "C:/Users/s1155063404/Desktop/WinBUGS14"

#Working directory
working_dir = "C:/Users/s1155063404/Desktop/Projects/brazilian-ecommerce-dataset/StatisticalAnalysis"

setwd(working_dir)
dataset = read_csv("./dataset.csv")

#Standardize the observations for modeling
standardize <- function(x){
  mean_x = mean(x)
  sig_x = sqrt(var(x))
  if (sig_x > 0){
    return((x - mean_x) / sig_x)
  }
  else {
    return(x)
  }
}

dataset$size = standardize(dataset$size)
dataset$order_products_value = standardize(dataset$order_products_value)
dataset$order_freight_value = standardize(dataset$order_freight_value)
dataset$distance = standardize(dataset$distance)

Parsed with column specification:
cols(
  size = col_double(),
  order_products_value = col_double(),
  order_freight_value = col_double(),
  distance = col_double(),
  cluster = col_integer(),
  customer_state = col_character(),
  customer_city = col_character(),
  delivery_time = col_double()
)


As I perform Bayesian Inference for the multi-level model, to have a fair comparison with the base model, I first perform Bayesian Inference for the base model. I use WinBUGS for Bayesian Inference, and I need to prepare the data and initial parameters for the WinBUGS program.

In [2]:
#Base model: Prepare the data, initial parameter for WinBUGS
K = nrow(dataset)
n_clust = length(unique(dataset$cluster))
Y = dataset$delivery_time
X = cbind(dataset$size, dataset$order_products_value, dataset$order_freight_value)
Z = dataset$distance
i = dataset$cluster

mu_b = rep(0, 3)
tau_b = diag(rep(0.0001, 3))

data = list(K=K, n_clust=n_clust, Y=Y, X=X, Z=Z, mu_b=mu_b, tau_b=tau_b, i=i)
bugs.data(data, dir="../WinBUGS_code/", data.file = "data_base.txt")

init = list(beta=rep(0,3), m1=0, m2=0, tau_a=10, tau_u=10, b=0.5)
para = c("m1", "beta", "m2", "sig_u", "sig_a", "b")

[1] "data_base.txt"

In [3]:
#Run WinBUGS for Bayesian Inference
sim = bugs(data="data_base.txt", inits=list(init),
           parameters.to.save=para,
           model.file="base.txt",
           n.chains=1, n.iter=1000,
           bugs.directory = WinBUGS_path,
           working.directory="../WinBUGS_code/")

In [4]:
sim

cbind(sim$mean, sim$sd)

Inference for Bugs model at "base.txt", fit using WinBUGS,
 1 chains, each with 1000 iterations (first 500 discarded)
 n.sims = 500 iterations saved
             mean     sd     2.5%      25%      50%      75%    97.5%
m1            2.6    0.0      2.6      2.6      2.6      2.6      2.6
beta[1]       0.0    0.0      0.0      0.0      0.0      0.0      0.0
beta[2]       0.0    0.0      0.0      0.0      0.0      0.0      0.0
beta[3]       0.0    0.0      0.0      0.0      0.0      0.0      0.0
m2            0.2    0.0      0.2      0.2      0.2      0.2      0.2
sig_u         0.1    0.0      0.1      0.1      0.1      0.1      0.1
sig_a         0.1    0.0      0.1      0.1      0.1      0.1      0.1
b             0.3    0.0      0.3      0.3      0.3      0.3      0.3
deviance 608231.8 1667.6 607300.0 607500.0 607500.0 607800.0 613910.0

DIC info (using the rule, pD = var(deviance)/2)
pD = 1390445.3 and DIC = 1998677.1
DIC is an estimate of expected predictive error (lower deviance is 

m1,2.596314,0.00372063
beta,"0.041022440, -0.004176734, 0.009921578","0.002223249, 0.002369514, 0.002146022"
m2,0.1972662,0.00303239
sig_u,0.12124,0.008267371
sig_a,0.09031932,0.007283062
b,0.2775242,0.00330493
deviance,608231.8,1667.6


## Feature Engineering (II):

Create second-level features for explaining the effects across customer cities. The second-level features are corresponding to each (state, city), thus I construct the following cluster table to store the features.

In [13]:
#Return the cluster label for each (state, city); Create the second-level features in this table.
cluster = dataset %>%
  group_by(customer_state, customer_city, cluster) %>%
  summarize(n = n()) %>%
  arrange(cluster)

I first compute the GDP rank for each underlying state to reflect the prosperity of city. The reason of not using GDP figure for each city is that, I cannot collect the GDP data for each single city (where the collected official data only gives the GDP figure of the top 100 cities). Thus, I use the GDP figure of the underlying state to reflect the prosperity effect, though it may not be the most appropriate figure. 

Moreover, I use the GDP rank instead of the GDP value because the scale of the GDP value of each state is vastly different.

In [14]:
raw_GDP_state = read_csv("../CleanedDataset/GDP_by_state_2015.csv")

GDP_state = raw_GDP_state %>%
  mutate(customer_state = Abbreviation) %>%
  select(customer_state, GDP_per_capita) %>%
  arrange(-GDP_per_capita)
GDP_state$rank = 1:nrow(GDP_state)

cluster = cluster %>%
  left_join(GDP_state, by="customer_state")

Parsed with column specification:
cols(
  Abbreviation = col_character(),
  State = col_character(),
  Capital = col_character(),
  Region = col_character(),
  GDP = col_character(),
  GDP_per_capita = col_double(),
  Capital_in_data = col_character()
)


I then compute the geolocation features of each city. I am interested in the latitude of the city, and the distance of the city to the most wealthy cities. I first estimate the location of each city.

In [15]:
raw_geolocation = read_csv("../CleanedDataset/corrected_geolocation.csv")

#geolocation of each city, state
geolocation = raw_geolocation %>%
  group_by(city, state) %>%
  summarize(lat = mean(lat),
            lng = mean(lng)) %>%
  ungroup() %>%
  mutate(customer_city = city,
         customer_state = toupper(state))

Parsed with column specification:
cols(
  zip_code_prefix = col_integer(),
  city = col_character(),
  state = col_character(),
  lat = col_double(),
  lng = col_double()
)


Then I compute the distance to closest 3 wealthy cities.

In [16]:
distance <- function(lat1, lng1, lat2, lng2){
  R = 6371 #Radius of Earth in km
  dang = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng2 - lng1)
  return(R * acos(pmax(pmin(dang, 1), -1)))
}
ang2rad <- function(ang){
  return(ang * pi / 180)
}

#Location of the 3 wealthy cities
location = geolocation %>%
  filter(city == "brasilia" & state == "df" |
           city == "sao paulo" & state == "sp" |
           city == "rio de janeiro" & state == "rj")

geolocation = geolocation %>%
  mutate(distance1 = distance(ang2rad(lat), ang2rad(lng),
                              ang2rad(as.numeric(location[1,3])), ang2rad(as.numeric(location[1,4]))),
         distance2 = distance(ang2rad(lat), ang2rad(lng),
                              ang2rad(as.numeric(location[2,3])), ang2rad(as.numeric(location[2,4]))),
         distance3 = distance(ang2rad(lat), ang2rad(lng),
                              ang2rad(as.numeric(location[3,3])), ang2rad(as.numeric(location[3,4])))) %>%
  mutate(distance2wealthy = pmin(distance1, distance2, distance3),
         distance1 = NULL,
         distance2 = NULL,
         distance3 = NULL)

#Add the geolocation features to the cluster table
cluster = cluster %>%
  left_join(geolocation %>%
              select(customer_state, customer_city, distance2wealthy, lat), by=c("customer_state", "customer_city"))

head(cluster)

customer_state,customer_city,cluster,n,GDP_per_capita,rank,distance2wealthy,lat
<chr>,<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>
MG,abadia dos dourados,1,4,24884.94,11,302.4037,-18.484990
GO,abadiania,2,2,26265.32,10,89.2244,-16.193984
MG,abaete,3,11,24884.94,11,457.6677,-19.158859
PA,abaetetuba,4,11,16009.98,22,1570.7397,-1.722619
CE,abaiara,5,1,14669.14,23,1351.5483,-7.361616
BA,abaira,6,1,16115.89,21,736.3914,-13.250568


Then I used the extracted features to build my multi-level model, using GLMM with Bayesian Inference. The procedure is similar as estimating the base model as above.

In [17]:
#Prepare the data
K = nrow(dataset)
n_clust = nrow(cluster)
Y = dataset$delivery_time
X = cbind(dataset$size, dataset$order_products_value, dataset$order_freight_value)
Z = dataset$distance
i = dataset$cluster

#Cluster_level features
W1 = standardize(cluster$rank)
W2 = standardize(cluster$lat)
W3 = standardize(cluster$distance2wealthy)

mu_b = rep(0, 3)
tau_b = diag(rep(0.0001, 3))
mu_nu = rep(0, 4)
tau_nu = diag(rep(0.0001, 4))

data = list(K=K, n_clust=n_clust, Y=Y, X=X, Z=Z, mu_b=mu_b, tau_b=tau_b, mu_nu=mu_nu, tau_nu=tau_nu,
            i=i, W1=W1, W2=W2, W3=W3)
bugs.data(data, dir="../WinBUGS_code/", data.file = "data_model.txt")

init = list(beta=rep(0,3), nu1=rep(0,4), nu2=rep(0,4), tau_a=10, tau_u=10, b=0.5)
para = c("beta", "nu1", "nu2", "sig_u", "sig_a", "b")

[1] "data_model.txt"

In [18]:
sim = bugs(data="data_model.txt", inits=list(init),
           parameters.to.save=para,
           model.file="model.txt",
           n.chains=1, n.iter=1000,
           bugs.directory=WinBUGS_path,
           working.directory="../WinBUGS_code/")

In [19]:
sim

cbind(sim$mean, sim$sd)

Inference for Bugs model at "model.txt", fit using WinBUGS,
 1 chains, each with 1000 iterations (first 500 discarded)
 n.sims = 500 iterations saved
             mean    sd     2.5%      25%      50%      75%    97.5%
beta[1]       0.0   0.0      0.0      0.0      0.0      0.0      0.0
beta[2]       0.0   0.0      0.0      0.0      0.0      0.0      0.0
beta[3]       0.0   0.0      0.0      0.0      0.0      0.0      0.0
nu1[1]        2.6   0.0      2.5      2.6      2.6      2.6      2.6
nu1[2]        0.1   0.0      0.1      0.1      0.1      0.1      0.2
nu1[3]       -0.1   0.0     -0.2     -0.1     -0.1     -0.1     -0.1
nu1[4]        0.0   0.0     -0.1     -0.1      0.0      0.0      0.0
nu2[1]        0.2   0.0      0.2      0.2      0.2      0.2      0.3
nu2[2]        0.0   0.0     -0.1      0.0      0.0      0.0      0.0
nu2[3]        0.0   0.0      0.0      0.0      0.0      0.0      0.1
nu2[4]        0.0   0.0      0.0      0.0      0.0      0.0      0.0
sig_u         0.1   0.

beta,"0.041714340, -0.004982019, 0.009232336","0.001696894, 0.001524649, 0.001451110"
nu1,"2.5889880, 0.1354806, -0.1198881, -0.0374774","0.04298235, 0.01668115, 0.04555743, 0.03633449"
nu2,"0.22385400, -0.04122352, 0.03547008, -0.01845796","0.021486553, 0.005627471, 0.016186739, 0.009609200"
sig_u,0.1123609,0.01046733
sig_a,0.07563312,0.01454196
b,0.2790798,0.001339822
deviance,607732.4,481.7618


Findings:

1. Based on the DIC figure, the multi-level model which uses the second-level features is statistically more appropriate, when compared with the base model.
2. Similar to the base model, the (intercept) coefficient of distance between seller and customer is the largest in magnitude when compared with other features, which indicates that the distance effect is the strongest feature here.
3. For the second-level feature, both GDP rank and the latitude of cities are considered significant features here, but the distance2wealthy feature is not significant. This is aline with my observation in the EDA.